In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
! pip install emoji
import emoji

# some NLP libraries that can help us with preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')

! pip install tweet_preprocessor
import preprocessor as tweet_preprocessor
! pip install twitter-text-python
from ttp import ttp
! pip install emoji
import emoji



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LoG1c\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LoG1c\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data.shape

(10600, 2)

In [4]:
def convert_to_emoji(text):
    return emoji.demojize(text)

In [5]:
tweets = data['tweet']
tweets_tokenized = []
tt = nltk.tokenize.TweetTokenizer()
for tweet in tweets:
    tweet = convert_to_emoji(tweet)
    # initializing punctuations string
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~"'?'''
 
    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in tweet:
        if ele in punc:
            tweet = tweet.replace(ele, "")
 
    tweets_tokenized.append(tt.tokenize(text=tweet))

In [6]:
tweets_tokenized

[['The',
  'CDC',
  'currently',
  'reports',
  '99031',
  'deaths',
  'In',
  'general',
  'the',
  'discrepancies',
  'in',
  'death',
  'counts',
  'between',
  'different',
  'sources',
  'are',
  'small',
  'and',
  'explicable',
  'The',
  'death',
  'toll',
  'stands',
  'at',
  'roughly',
  '100000',
  'people',
  'today'],
 ['States',
  'reported',
  '1121',
  'deaths',
  'a',
  'small',
  'rise',
  'from',
  'last',
  'Tuesday',
  'Southern',
  'states',
  'reported',
  '640',
  'of',
  'those',
  'deaths',
  'httpstcoYASGRTT',
  '4ux'],
 ['Politically',
  'Correct',
  'Woman',
  'Almost',
  'Uses',
  'Pandemic',
  'as',
  'Excuse',
  'Not',
  'to',
  'Reuse',
  'Plastic',
  'Bag',
  'httpstcothF',
  '8GuNFPe',
  'coronavirus',
  'nashville'],
 ['IndiaFightsCorona',
  'We',
  'have',
  '1524',
  'COVID',
  'testing',
  'laboratories',
  'in',
  'India',
  'and',
  'as',
  'on',
  '25th',
  'August',
  '2020',
  '3682752',
  '0',
  'tests',
  'have',
  'been',
  'done',
  'Pro

In [7]:
#remove stopwords
stemmer = nltk.stem.porter.PorterStemmer()
stoplist = nltk.corpus.stopwords.words('english')

stemmed = []

for tweet in tweets_tokenized:
    processed_tokens = [stemmer.stem(t) for t in tweet if t not in stoplist]
    stemmed.append(processed_tokens)


In [8]:
def super_simple_preprocess(text):
  # lowercase
  text = text.lower()
  # remove non alphanumeric characters
  text = re.sub('[^A-Za-z0-9 ]+','', text)
  return text

In [9]:
pd.DataFrame(stemmed).shape

(10600, 1113)

In [10]:
data_updated = []
for tweet in stemmed:
    data_updated.append(super_simple_preprocess(" ".join(tweet)))
pd.DataFrame(data_updated).shape

(10600, 1)

In [11]:
data_updated = pd.DataFrame(data_updated)
data_updated.insert(1, 'w', data['label'])
data = data_updated

In [12]:
from sklearn.model_selection import train_test_split
X = data.iloc[:, 0]
y = data.iloc[: , 1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train : ", len(X_train),"Test : ", len(X_test),"Val : ", len(X_val))

def to_csvs(X_train,y_train,X_test,y_test,X_val,y_val):
    X_train = pd.DataFrame(X_train)
    X_train.insert(1, 'label', y_train)
    X_train.to_csv('train.csv')
    X_test = pd.DataFrame(X_test)
    X_test.insert(1, 'label', y_test)
    X_test.to_csv('test.csv')
    X_val = pd.DataFrame(X_val)
    X_val.insert(1, 'label', y_val)
    X_val.to_csv('val.csv')

to_csvs(X_train,y_train,X_test,y_test,X_val,y_val)

Train :  8480 Test :  1060 Val :  1060


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=150000)
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)

In [14]:
X_train_text.shape, X_test_text.shape

((8480, 107479), (1060, 107479))

In [15]:
import pickle
with open('train_vectorized.pickle', 'rb') as file:
    X_train_text = pickle.load(file)

with open('test_vectorized.pickle', 'rb') as file:
    X_test_text = pickle.load(file)

with open('train.pickle', 'rb') as file:
    X_train = pickle.load(file)

with open('test.pickle', 'rb') as file:
    X_test = pickle.load(file)

In [16]:
with open('train_vectorized.pickle', 'rb') as file:
    X_train_text = pickle.load(file)

with open('test_vectorized.pickle', 'rb') as file:
    X_test_text = pickle.load(file)

with open('train.pickle', 'rb') as file:
    X_train = pickle.load(file)

with open('test.pickle', 'rb') as file:
    X_test = pickle.load(file)

y_train = X_train.iloc[:,1]
y_test = X_test.iloc[:, 1]

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score, classification_report
logit = LogisticRegression(C=3.34431505414951, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4, penalty='l2')
logit.fit(X_train_text, y_train)
y_pred = logit.predict(X_test_text)

In [18]:
print('classification_report : ', classification_report(y_test, y_pred))
print('confusion matrix : ', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy score :' , accuracy_score(y_test, y_pred))
recall_average = recall_score(y_test, y_pred, average="binary", pos_label='real')
print("recall_average : ", recall_average)
precision_average = precision_score(y_test, y_pred, average="binary", pos_label='real')
print("precision_average : ", precision_average)
f1_average = f1_score(y_test, y_pred, average="binary", pos_label='real')
print("f1_score : ", f1_average)

classification_report :                precision    recall  f1-score   support

        fake       0.91      0.94      0.93       501
        real       0.95      0.92      0.93       559

    accuracy                           0.93      1060
   macro avg       0.93      0.93      0.93      1060
weighted avg       0.93      0.93      0.93      1060

confusion matrix :  [[472  29]
 [ 47 512]]
accuracy score : 0.9283018867924528
recall_average :  0.9159212880143113
precision_average :  0.9463955637707948
f1_score :  0.9309090909090909


In [19]:
from sklearn.svm import SVC
svc = SVC(C=1, kernel='rbf')
svc.fit(X_train_text, y_train)
y_pred = svc.predict(X_test_text)
print('confusion matrix : ', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy score :' , accuracy_score(y_test, y_pred))
recall_average = recall_score(y_test, y_pred, average="binary", pos_label='real')
print("recall_average : ", recall_average)
precision_average = precision_score(y_test, y_pred, average="binary", pos_label='real')
print("precision_average : ", precision_average)
f1_average = f1_score(y_test, y_pred, average="binary", pos_label='real')
print("f1_score : ", f1_average)

confusion matrix :  [[471  30]
 [ 45 514]]
accuracy score : 0.9292452830188679
recall_average :  0.9194991055456172
precision_average :  0.9448529411764706
f1_score :  0.9320036264732547


from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train_text, y_train)
y_pred = clf.predict(X_test_text)
print('accuracy', clf.score(X_test_text, y_test))

from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C':[1, 10, 100, 1000, 0.1], 'gamma':('scale', 'auto')}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train_text, y_train)
print(clf.cv_results_)

clf.best_estimator_
y_pred = clf.predict(X_test_text)

In [ ]:
print('confusion matrix : ', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy score :' , accuracy_score(y_test, y_pred))
recall_average = recall_score(y_test, y_pred, average="binary", pos_label='real')
print("recall_average : ", recall_average)
precision_average = precision_score(y_test, y_pred, average="binary", pos_label='real')
print("precision_average : ", precision_average)
f1_average = f1_score(y_test, y_pred, average="binary", pos_label='real')
print("f1_score : ", f1_average)

confusion matrix :  [[475  26]
 [ 41 518]]
accuracy score : 0.9367924528301886
recall_average :  0.9266547406082289
precision_average :  0.9522058823529411
f1_score :  0.9392565729827741


In [21]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(X_train_text)
y_pred = kmeans.predict(X_test_text)

In [22]:
y_pred

array([1, 1, 2, ..., 2, 2, 0])

In [23]:
pd.DataFrame(y_pred).value_counts()

2    591
1    371
0     98
Name: count, dtype: int64

In [24]:
y_test

4395     real
7786     real
6204     fake
6133     fake
4822     real
         ... 
6718     fake
6734     real
7672     real
9863     real
10434    real
Name: label, Length: 1060, dtype: object

In [26]:
k_values = [2, 3, 4, 5, 6]

best_accuracy = 0
best_k = None

for k in k_values:
    # Fit KMeans model
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_train_text)
    
    # Assign cluster labels
    cluster_labels = kmeans.labels_
    
    # Determine majority label for each cluster
    majority_labels = []
    for i in range(k):
        cluster_indices = np.where(cluster_labels == i)[0]
        majority_label = np.argmax(np.bincount(y_train[cluster_indices]))
        majority_labels.append(majority_label)

    
    
    # Assign predicted labels based on majority labels
    y_pred = [majority_labels[label] for label in cluster_labels]
    
    # Calculate accuracy on test set
    accuracy = accuracy_score(y_test, y_pred)
    
    # Update best accuracy and k value
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k

print("Best k:", best_k)
print("Best accuracy:", best_accuracy)


C:\Users\LoG1c\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyError: '[0, 2, 4, 7, 12, 14, 23, 27, 28, 36, 44, 55, 56, 61, 64, 65, 75, 78, 80, 102, 113, 121, 137, 148, 152, 163, 169, 174, 177, 178, 181, 185, 189, 194, 195, 196, 200, 202, 205, 207, 213, 216, 220, 222, 223, 228, 233, 235, 255, 259, 260, 262, 263, 270, 273, 278, 283, 302, 314, 315, 320, 341, 357, 360, 364, 370, 374, 375, 391, 393, 396, 413, 415, 417, 422, 424, 429, 432, 439, 443, 446, 448, 449, 464, 465, 479, 480, 492, 507, 509, 515, 520, 524, 532, 534, 542, 546, 547, 549, 550, 557, 563, 565, 567, 596, 597, 598, 599, 608, 612, 629, 643, 648, 651, 661, 668, 669, 672, 674, 676, 677, 681, 685, 690, 713, 720, 724, 739, 756, 763, 764, 771, 773, 780, 787, 790, 797, 802, 810, 817, 825, 826, 830, 831, 833, 836, 853, 860, 872, 879, 887, 888, 900, 903, 908, 910, 913, 914, 918, 919, 927, 930, 944, 945, 947, 949, 952, 959, 975, 978, 988, 991, 993, 1010, 1011, 1014, 1019, 1027, 1028, 1029, 1035, 1044, 1050, 1054, 1056, 1057, 1063, 1100, 1117, 1131, 1132, 1134, 1136, 1139, 1141, 1149, 1154, 11

In [ ]:
def kmeanstry():
    for n in range(2,10,2):
        kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(X_train_text)
        y_pred = kmeans.predict(X_test_text)

In [29]:
y_pred1 = []
for l in y_pred:
    if l == 0:
        y_pred1.append('fake')
    else:
        y_pred1.append('real')

y_pred = y_pred1



print('confusion matrix : ', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy score :' , accuracy_score(y_test, y_pred))
recall_average = recall_score(y_test, y_pred, average="binary", pos_label='real')
print("recall_average : ", recall_average)
precision_average = precision_score(y_test, y_pred, average="binary", pos_label='real')
print("precision_average : ", precision_average)
f1_average = f1_score(y_test, y_pred, average="binary", pos_label='real')
print("f1_score : ", f1_average)

confusion matrix :  [[276 203]
 [333 248]]
accuracy score : 0.49433962264150944
recall_average :  0.4268502581755594
precision_average :  0.549889135254989
f1_score :  0.48062015503875966


In [22]:

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train_text, y_train)
y_pred = neigh.predict(X_test_text)

print('confusion matrix : ', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy score :' , accuracy_score(y_test, y_pred))
recall_average = recall_score(y_test, y_pred, average="binary", pos_label='real')
print("recall_average : ", recall_average)
precision_average = precision_score(y_test, y_pred, average="binary", pos_label='real')
print("precision_average : ", precision_average)
f1_average = f1_score(y_test, y_pred, average="binary", pos_label='real')
print("f1_score : ", f1_average)

confusion matrix :  [[455  46]
 [ 66 493]]
accuracy score : 0.8943396226415095
recall_average :  0.8819320214669052
precision_average :  0.9146567717996289
f1_score :  0.8979963570127504


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_neighbors': [2,3,4,5,10,15,20,25,30], 'weights' : ('uniform', 'distance'), 
              'algorithm' : ('auto', 'ball_tree', 'kd_tree', 'brute')}
neigh = KNeighborsClassifier()
clf = GridSearchCV(neigh, parameters)
clf.fit(X_train_text, y_train)
print(clf.best_estimator_)

In [31]:
data.head()
data['combined'] = "__label__" + data['w'] + " " + data.iloc[:, 0]

In [32]:
train, test = train_test_split(data, test_size=0.2)

In [33]:
train.shape, test.shape

((8480, 3), (2120, 3))

In [34]:
test, val = train_test_split(test, test_size=0.5)

In [35]:
test.shape, val.shape

((1060, 3), (1060, 3))

In [36]:
train.head()

,0,w,combined
4464,tucker carlson claim coronaviru pose virtual ...,fake,__label__fake tucker carlson claim coronaviru ...
9002,in india congress presid approv requir use fu...,fake,__label__fake in india congress presid approv...
2400,our estim suggest effect covid 19 vaccin distr...,real,__label__real our estim suggest effect covid 1...
9464,indiafightscorona centr exhort north eastern s...,real,__label__real indiafightscorona centr exhort n...
2442,loudspeakercoronavirusupd checkmarkbuttonindia...,real,__label__real loudspeakercoronavirusupd checkm...


In [37]:
train.to_csv('data.train', columns=['combined'], index=False, header=False)
train.to_csv('data.test', columns=['combined'], index=False, header=False)
train.to_csv('data.val', columns=['combined'], index=False, header=False)

In [38]:
import fasttext

model = fasttext.train_supervised(input='data.train')
model.test('data.test')

(8480, 0.9826650943396227, 0.9826650943396227)